In [1]:
from oot.oot import Oot
import os, random, requests, time
import xlrd
import pandas as pd

ACTOR_OVERLAY_TABLE = 0x00B5E490 
AOT_SIZE = 470 * 32 # Each record is 32 bytes long
OBJECT_REC_SIZE = 2 # In Bytes
ACTOR_REC_SIZE = 16 # In Bytes
CACHE_ENABLED = True
SEED = None
LOGDIR = 'logs'
GAMEDIR = 'roms'

In [2]:
# General helper and debug methods
toHex = lambda b: hex(b)[2:].zfill(2).upper()
def make_readable(data):
    # data is a bytearray
    # return a string with the data in a format I can easily read
    l = [] # list of byte words 
    for i in range(0, len(data), 4):
        l.append(''.join([toHex(b) for b in data[i:i+4]]))
    if i < len(data)-4:
        l.append(''.join([toHex(b) for b in data[i:]]))
    formatted_list = []
    for b in range(0, len(l), 4):
        avail_bytes = l[b:b+4]
        string = ' '.join(['{}' for b in avail_bytes])
        formatted_list.append(string.format(*avail_bytes))
    return '\n'.join(formatted_list)
def setSeed(seed):
    random.seed(seed)
def writeSpoilers(spoilers, fn="spoiler.log"):
    fn = os.path.join(LOGDIR,fn)
    # Write out a spoiler object
    s = ''
    for room in spoilers:
        s += '{}\n'.format(room)
        for replacement in spoilers[room]:
            old = replacement["old"]
            new = replacement["new"]
            s +='{}({})[{}] -> {}({})[{}]\n'.format(old["filename"],old["description"],old["variable"],new["filename"],new["description"],new["variable"])
    with open(fn, "w") as f:
        f.write(s)

In [3]:
# Create reference tables
fn = "data.xlsx"
if not os.path.isfile(fn):
    req = requests.get('https://drive.google.com/uc?authuser=0&id=1kjLzvYSLfFtsRoVEtSMwpmZEcwCSEJIX&export=download')
    with open(fn, "wb") as f:
        f.write(req.content)

book = xlrd.open_workbook("data.xlsx")
def sheetToPandas(name):
    sheet = book.sheet_by_name(name)
    table = [[c.value for c in r] for r in sheet.get_rows()]
    table_headers = table[0]
    table_data = table[1:]
    return pd.DataFrame(table_data, columns=table_headers)
ref_files = sheetToPandas('files').set_index('Filename')
ref_actors = sheetToPandas('actors').set_index('Record No')
ref_objects = sheetToPandas('objects').set_index('Record No')
ref_enemies = sheetToPandas('enemies').set_index('Enemy')
#ref_entrances = sheetToPandas('entrances')
# Caches for various things
actor_object_cache = {} # Caches which object is associated with an actor

In [4]:
# Define methods for interacting with the ROM
game = Oot(gamedir=GAMEDIR, logdir=LOGDIR)
def readData():
    game.readData()
def getData(start, end=None, to_end=False, size=None):
    if to_end:
        return game.data[start:]
    if not end:
        if size:
            end = start+size
        else:
            return game.data[start]
    return game.data[start:end]
def setData(addr, barr):
    game.set_bytes(addr, barr)
def writeData():
    game.create()
def searchData(st):
    # Search the ROM for instances of the above string of hex bytes, with * as a wildcard character
    # Return a list of hex addresses that match
    # '01 B9 FE B6 01 E0 F7 04'
    foundInsts = []
    bytelist = [int(b,16) if b != '*' else '*' for b in st.split(' ')]
    for b in range(len(game.data)):
        instFound = True
        for c in range(len(bytelist)):
            if bytelist[c] == '*' or bytelist[c] == game.data[b+c]:
                continue
            instFound = False
            break
        if instFound:
            foundInsts.append(hex(b))
    return foundInsts

Using seed 1545859402


In [5]:
# Some methods for looking up information in the reference tables
def lookupObjectDescription(fn):
    x = ref_objects[ref_objects['Filename'] == fn]['Description'][0]
    # I think this is necessary
    if type(x) == pd.core.series.Series:
        x = x.iloc[0] # There's only two object duplicates
    return x
def lookupObjectNum(fn):
    x = ref_objects[ref_objects['Filename'] == fn].index[0] # Get the record No
    if type(x) == pd.core.series.Series:
        x = x.iloc[0] # There's only two object duplicates
    return x
def lookupActorNum(fn):
    return ref_actors[ref_actors['Filename'] == fn].index[0] # Get the record No
def lookupRoomHeader(addr, isHex=True):
    dec_addr = int(addr, 16) if isHex else addr 
    return Header(dec_addr)
def getRoomAddrByName(name):
    record = ref_files.loc[name]
    return record['VROM Start']
def getRoomHeaderByName(name):
    return lookupRoomHeader(getRoomAddrByName(name))
def createRoomByName(name):
    record = ref_files.loc[name]
    return Room(record['VROM Start'], name, record['Contents'])
def getRoomNames():
    df_filenames = ref_files[ref_files['Description'] == 'Scenes/Rooms'].index
    return [r for r in df_filenames if "_room_" in r]  # We don't want the scenes
def generateRooms(names):
    return {n: createRoomByName(n) for n in names}
def replaceActorInRoom(old, new, room_name, var='0000', old_object_fn=None, new_object_fn=None, index=None):
    createRoomByName(room_name).replaceActor(old,new, var, old_object_fn=old_object_fn, new_object_fn=new_object_fn, index=index)
def listActorInformation(fn):
    found_info = {}
    rooms = generateRooms(getRoomNames())
    for name, room in rooms.items():
        for num, setup in room.setups.items():
            actors = [a for a in setup['actors'] if a.filename == fn]
            if len(actors) > 0:
                found_info['{}-{} ({})'.format(name, num, room.descr)] = actors
    for room, actors in found_info.items():
        print(room)
        for actor in actors:
            print(actor.getInfo())
def spawnAt(scene, entrance=0x0):
    # Changes the spawn location for kid link (out of dungeons) to the specified record no (as a string)
    ent_table = 0x00B6FBF0 # Start of entrance table
    links_house_ent = ent_table+(4*0xBB) # Record No of the entrance to 
    setData(links_house_ent, [scene, entrance])

In [6]:
# Define the classes used
class Header():
    def __init__(self, addr):
        self.start_addr = addr
        self.end_addr = self.findEndAddr()
        self.header = getData(self.start_addr, self.end_addr)
        self.objectListDef = self.getObjectListDef()
        self.actorListDef = self.getActorListDef()
    def findEndAddr(self):
        curr_addr = self.start_addr
        while getData(curr_addr) != int(0x14):
            curr_addr += 8
        return curr_addr + 8
    def getObjectListDef(self):
        defn = self.lookupCommand('0B')
        if not defn:
            return {"num": 0, "offset": 0}
        return {"num": defn[1], "offset": int(''.join([toHex(b) for b in defn[-3:]]),16)}
    def getActorListDef(self):
        defn = self.lookupCommand('01')
        if not defn:
            return {"num": 0, "offset": 0}
        return {"num": defn[1], "offset": int(''.join([toHex(b) for b in defn[-3:]]),16)}
    def getAlternateHeaders(self, limit=3):
        # Limit is 3 because afaik there are 4 available scenes 
        c = self.lookupCommand('18')
        if not c: # Some rooms don't have any alternate headers (e.g. dungeons)
            return None
        list_addr = self.start_addr + int(''.join([str(i) for i in c[5:]]))
        header_addresses = []
        for i in range(limit):
            addr = getData(list_addr+(i*4), size=4)
            if addr[0] != 0x03:
                if int(''.join([str(i) for i in addr]),16) == 0:
                    continue # Offset of 0 means it uses the original header
                raise Exception("Adress Segment is not 3, I probably wouldn't handle it correctly")
            header_addresses.append(self.start_addr + int(''.join([toHex(i) for i in addr[1:]]), 16))
        return header_addresses 
    def lookupCommand(self, num, isHex=True):
        dec_num = int(num, 16) if isHex else num
        possibleCommands = []
        for b in range(0, len(self.header), 8): # Each command is 8 bytes
            if self.header[b] == dec_num:
                possibleCommands.append(self.header[b:b+8])
        if len(possibleCommands) < 1:
            return None
        if len(possibleCommands) > 1:
            raise NotImplementedError("Looking up nonunique commands not yet implemented")
        return possibleCommands[0]
class Actor():
    def __init__(self, num, offset, var=0):
        self.num = num
        self.var = var
        self.offset = offset # What offset is the actor in the actor list
        self.filename = self.lookupFilename()
        self.object_name, self.description = self.matchObject()
    def getInfo(self):
        return "{}) {} {} ({}) [{}] -> {}".format(self.offset, self.num, self.filename, self.description, self.var, self.object_name)
    def lookupFilename(self):
        return ref_actors.loc[self.num].Filename
    def lookupInitVars(self):
        return ref_actors.loc[self.num]['Init Vars']
    def lookupRAMStart(self):
        return ref_actors.loc[self.num]['VRAM Start']
    def lookupROMStart(self):
        return ref_actors.loc[self.num]['VROM Start']
    def matchObject(self):
        if CACHE_ENABLED and self.filename in actor_object_cache:
            return actor_object_cache[self.filename]["name"], actor_object_cache[self.filename]["description"]
        else:
            if self.filename in ['En_A_Obj', 'En_Item00']:
                objId = '0001' # These objects need a special case
            else:
                objId_address = int(self.lookupROMStart(),16) + int(self.lookupInitVars(),16) - int(self.lookupRAMStart(),16) + 8
                objId = ''.join([toHex(b) for b in getData(objId_address, objId_address +2)])
            if objId not in ref_objects.index:
                return "NA", ""
            row = ref_objects.loc[objId]
            info = {"name": row['Filename'], "description":row['Description']}
            actor_object_cache[self.filename] = info
        return info["name"], info["description"]
    def setObject(self, fn):
        if fn in actor_object_cache:
            object_info = actor_object_cache[fn]
        else:
            object_info = {"name": fn, "description": lookupObjectDescription(fn)}
        self.object_name = object_info["name"]
        self.description = object_info["description"]
class ZObject(): # This object is related to the object instance in a room, not to a specific kind of object
    def __init__(self, num, offset):
        self.objId = num
        self.offset = offset # What offset is the object in the object list
        self.filename = self.lookupFilename()
        self.description = self.lookupDescription()
    def getInfo(self):
        return "{}) {} {} ({})".format(self.offset, self.objId, self.filename, self.description)
    def lookupFilename(self):
        return ref_objects.loc[self.objId].Filename
    def lookupDescription(self):
        return ref_objects.loc[self.objId].Description
class Room():
    def __init__(self, addr, fn, descr=''):
        self.address = addr
        self.fn = fn
        self.descr = descr
        self.setups = {}
        self.setups[0] = self.getSceneSetup(lookupRoomHeader(self.address))
        alternateSetupAddrs = self.setups[0]['header'].getAlternateHeaders()
        if alternateSetupAddrs:
            for i, addr in enumerate(alternateSetupAddrs):
                self.setups[i+1] = self.getSceneSetup(lookupRoomHeader(addr, isHex=False))
    def getSceneSetup(self,header):
        object_list_addr = int(self.address, 16) + header.objectListDef['offset']
        actor_list_addr = int(self.address, 16) + header.actorListDef['offset']
        return {
            "header": header,
            "object_list_addr": object_list_addr,
            "objects": self.getObjects(object_list_addr, header.objectListDef['num']),
            "actor_list_addr": actor_list_addr,
            "actors": self.getActors(actor_list_addr, header.actorListDef['num'])
        }
    def getActors(self, addr, nactors):
        actor_list_data = getData(addr,addr+(nactors*ACTOR_REC_SIZE))
        actor_list = []
        for a in range(0, len(actor_list_data), ACTOR_REC_SIZE):
            var_offset = 14 # how many bytes into the actor_list_data is the variable
            actor_num = ''.join([toHex(b) for b in actor_list_data[a:a+2]])
            variable = ''.join([toHex(b) for b in actor_list_data[a+var_offset:a+var_offset+2]])
            actor = Actor(actor_num, a // ACTOR_REC_SIZE, var=variable)
            actor_list.append(actor)
        return actor_list
    def getObjects(self, addr, numobjects):
        object_list_data = getData(addr,addr+(numobjects*OBJECT_REC_SIZE))
        object_list = []
        for o in range(0, len(object_list_data), OBJECT_REC_SIZE):
            objId = ''.join([toHex(b) for b in object_list_data[o:o+2]])
            obj = ZObject(objId, o // OBJECT_REC_SIZE)
            object_list.append(obj)
        return object_list
    def hasObject(self, fn, setup):
        return fn in [o.filename for o in self.setups[setup]['objects']]
    def replaceObject(self, old_fn, new_fn, setup=0):
        roomSetup = self.setups[setup]
        for z in range(len(roomSetup['objects'])):
            zobj = roomSetup['objects'][z]
            if old_fn == zobj.filename:
                obj_addr = roomSetup['object_list_addr'] + zobj.offset*OBJECT_REC_SIZE
                newObj = ZObject(lookupObjectNum(new_fn), zobj.offset)
                setData(obj_addr, [int(newObj.objId[:2],16), int(newObj.objId[2:],16)])
                roomSetup['objects'][z] = newObj
    def replaceActor(self, old_fn, new_fn, new_var='0000', old_object_fn=None, new_object_fn=None, index=None, setup=0, replaceObject=True, absIndex=None):
        c = -1
        if not setup in self.setups:
            raise Exception("Trying to replace actor in nonexistent setup!")
        roomSetup = self.setups[setup]
        for a in range(len(roomSetup['actors'])):
            if absIndex != None and a != absIndex:
                continue
            actor = roomSetup['actors'][a]
            if actor.filename == old_fn:
                if index != None:
                    c += 1
                    if index != c:
                        continue
                actor_addr = roomSetup['actor_list_addr'] + actor.offset*ACTOR_REC_SIZE
                # lookupActor should be used by getActors
                newActor = Actor(lookupActorNum(new_fn), actor.offset, new_var)
                if old_object_fn:
                    actor.setObject(old_object_fn)
                if new_object_fn:
                    newActor.setObject(new_object_fn)
                if actor.object_name == 'NA' and replaceObject:
                    raise Exception("Old Actor's Corresponding Object can't be identified")
                if newActor.object_name == 'NA' and replaceObject:
                    raise Exception("New Actor's Corresponding Object can't be identified")
                # This seems stupid, why am I converting the num to hex to convert back to dec
                # Should be a method in the Actor class for setting the data to what's stored in the class
                setData(actor_addr, [int(newActor.num[:2], 16), int(newActor.num[2:], 16)])
                var_offset = 14 # how many bytes into the actor_list_data is the variable
                setData(actor_addr+var_offset, [int(newActor.var[:2], 16), int(newActor.var[2:], 16)])
                def _associatedWithBaseObject(actor):
                    return actor.object_name in ['gameplay_keep', 'gameplay_field_keep', 'gameplay_dangeon_keep']
                if replaceObject and not _associatedWithBaseObject(newActor) and not self.hasObject(newActor.object_name, setup):
                    self.replaceObject(actor.object_name, newActor.object_name, setup=setup)
                roomSetup['actors'][a] = newActor
                if index != None:
                    return
    def getInfo(self, setupNum=None):
        def _getInfoForSetup(num):
            global x
            x = self.setups[num]
            return '\n'.join([
                '{} ({}) setup {} @ {}'.format(self.fn, self.descr, num, self.setups[num]['header'].start_addr),
                '### Objects ###',
                *[o.getInfo() for o in self.setups[num]['objects']],
                '### Actors ###',
                *[a.getInfo() for a in self.setups[num]['actors']]
            ])
        if not setupNum:
            return '\n\n'.join([_getInfoForSetup(i) for i in self.setups])
        return self.setups[setupNum]

In [ ]:
# Cell for testing individual actor replacements
# Kokiri main spot04_room_0
# To replace rock guy old_actor=En_Ko, old_object_fn=object_km1
# Mido's House kokiri_home4_room_0
readData()
replaceActorInRoom('En_Tite', 'En_Mb', 'MIZUsin_room_0', var='00FF')
#replaceActorInRoom('En_Box', 'En_Box', 'kokiri_home4_room_0', var='0700')
#replaceActorInRoom('En_Ko', 'En_Rr', 'spot04_room_0', var='0003', index=0, old_object_fn='object_os_anime')
# replaceActorInRoom('En_Karebaba', 'En_Skj', 'spot04_room_1', var='FFFF', old_object_fn='object_dekubaba')
spawnAt(0x5)
writeData()

In [ ]:
new_actor[0]

In [ ]:
new_var

In [ ]:
choice[1][0][0]

In [ ]:
# Running this cell should start to finish randomize the actors in the rom
actor_prefix = 'En_Wf' # Only randomize among actors starting with this prefix
start_time = time.time()
seed = SEED if SEED else str(start_time)
setSeed(seed)
print("Seed: {}".format(seed) )

readData()
print("Data read, {} sec".format(time.time()-start_time))

rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
print("Room Info Generated, {} sec".format(time.time()-start_time))

# Generate a dict with actor/{object: associated_object, variable: var} pairs
actors_associated = {}
for key, room in rooms.items():
    for num, setup in room.setups.items():
        for actor in setup['actors']:
            if actor.object_name is not 'NA':
                actors_associated[actor.filename] = {
                    "object": actor.object_name,
                    "variable": actor.var
                }
available_actors = {k:a for k,a in actors_associated.items() if k.startswith(actor_prefix)}
print("Actors Associated, {} sec".format(time.time()-start_time))

spoilers = {}
for r, room in rooms.items():
    for n, setup in room.setups.items():
        key = '{}-{}'.format(r,n)
        room_spoiler = []
        spoilers[key] = room_spoiler
        for a, actor in enumerate(setup['actors']):
            if actor.filename in available_actors and actor.filename not in [rm["new"]["filename"] for rm in room_spoiler]:
                new_actor_name = random.choice(list(available_actors.keys()))
                old_actor_info = {"filename": actor.filename, "object": actor.object_name, "description": actor.description, "variable": actor.var}
                room.replaceActor(actor.filename, new_actor_name, new_var=available_actors[new_actor_name]['variable'])
                new_actor_info = {"filename": setup['actors'][a].filename, "object": setup['actors'][a].object_name, "description": setup['actors'][a].description, "variable": actor.var}
                room_spoiler.append({"old": old_actor_info, "new": new_actor_info})
        spoilers[r] = room_spoiler
print("Rooms Randomized, {} sec".format(time.time()-start_time))

# Print out spoilers
writeSpoilers(spoilers, "spoiler.log")
print("Replacement Log Written".format(time.time()-start_time))

writeData()
print("New ROM Written {}".format(time.time()-start_time))

In [ ]:
# Running this cell should start to finish randomize the actors within each setup
start_time = time.time()
seed = SEED if SEED else str(start_time)
setSeed(seed)
print("Seed: {}".format(seed) )

readData()
print("Data read, {} sec".format(time.time()-start_time))

rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
print("Room Info Generated, {} sec".format(time.time()-start_time))
        
def _writeSpoilers(sp, fn):
    with open(fn, "w") as f:
        for k,v in sp.items():
            f.write('{}\n'.format(k))
            for c in v:
                f.write('{} ({}) -> {} ({})\n'.format(c['old']['fn'], c['old']['var'], c['new']['fn'], c['new']['var']))
spoilers = {}
for r, room in rooms.items():
    for n, setup in room.setups.items():
        key = '{}-{}'.format(r,n)
        room_spoiler = []
        spoilers[key] = room_spoiler
        setup_actors = [ {"fn": a.filename, "var": a.var}
           for a in setup['actors']]
        new_setup_actors = list(setup_actors)
        random.shuffle(new_setup_actors)
        for a, actor in enumerate(setup['actors']):
            old = setup_actors[a]
            new = new_setup_actors[a]
            room.replaceActor(old['fn'], new['fn'], new_var=new['var'], absIndex=a, replaceObject=False, setup=n)
            room_spoiler.append({'old': old, 'new': new})
print("Rooms Randomized, {} sec".format(time.time()-start_time))

# Print out spoilers
_writeSpoilers(spoilers, "spoiler.log")
print("Replacement Log Written".format(time.time()-start_time))

writeData()
print("New ROM Written {}".format(time.time()-start_time))

In [7]:
### Cell to randomize enemies ONLY
start_time = time.time()
SEED = 99991542915092.487943#'Winnie'
seed = SEED if SEED else str(start_time)
setSeed(seed)
print("Seed: {}".format(seed) )
selected_enemy = None#{'fn': 'En_Torch2', 'object_fn': 'object_torch2', 'var': ('FFFF', '')} # Mostly for debug will change all enemies to this type

def _readVars(string):
    if len(string) < 2:
        return []
    return map(lambda x: x.strip(), string.split(","))

enemies = {}
for enemy in ref_enemies.index[1:]:
    enemy_info = ref_enemies.loc[enemy]
    if enemy_info['Enabled']:
        key = enemy_info['Requirements']
        if enemy_info['Requirements'] not in enemies:
            enemies[key] = {}
        # Variables are of the form 'XXXX (), '
        variables = enemy_info['Variable\'s'].split(',')
        variables = [v.split(' (') for v in variables]
        real_variables = []
        for v in variables:
            var = v[0].strip()
            if len(var) != 4: 
                raise Exception("Enemy {} has invalid variable {}".format(enemy, list(var)))
            descr = ''
            if len(v) > 1:
                descr = v[1].split(')')[0]
            real_variables.append((var,descr))
        # From Variables just a list of variables rather than having the description too
        from_variables = enemy_info['From-Var\'s']
        if len(from_variables) > 0:
            print(from_variables)
        enemies[key][enemy] = {
            "actor_fn": enemy_info['Actor FN'],
            "object_fn": enemy_info['Object FN'],
            "variables": real_variables,
            "from_variables": from_variables,
            "descr": descr,
            "type": key,
            "enemy": enemy
        }
        
available_actors = list(ref_enemies['Actor FN'].values)
available_objects = list(ref_enemies['Object FN'].values)

def isEnemyObject(st):
    return st in available_objects
def isEnemyActor(st):
    return st in available_actors
        
object_enemies = { k: [e["object_fn"] for e in v.values()] for k,v in enemies.items() }

actor_var_pairs = {}
from_actor_var_pairs = {}
for t,el in enemies.items():
    for e in el.values():
        for var in e["variables"]:
            actor_var_pairs['{}-{}'.format(e['actor_fn'], var[0])] = e
        for var in _readVars(e["from_variables"]):
            from_actor_var_pairs['{}-{}'.format(e['actor_fn'], var[0])] = e
print("enemy-type lists created: {}".format(time.time()-start_time))

readData()
print("Data read, {} sec".format(time.time()-start_time))

rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
print("Room Info Generated, {} sec".format(time.time()-start_time))

# I think I can just assume that a object is only related to one actor in a room

# Once I have every actor linked to an object, it will be easier to have a way to look and see what actors are associated with an object
myx = None # Debug variable
def _getObjType(obj, actors, room):
    global myx
    # Find the actor that is associated with the object, and return that actors type
    for actor in actors:
        if actor.object_name == obj.filename:
            n = '{}-{}'.format(actor.filename,actor.var)
            if not n in actor_var_pairs:
                myx = (actor, obj, room, n)
                if not n in from_actor_var_pairs:
                    return False # don't randomize
                else:
                    return actor_var_pairs[n]['type'] # I think this works because the from vars enemies aren't in the main pool
            return actor_var_pairs[n]['type']
    return False#'dont-randomize' # object doesn't have a corresponding actor so it's probably dynamically generated. so don't randomize

def _getActorType(actor):
    return actor_var_pairs['{}-{}'.format(actor.filename, actor.var)]['type']
def _getObjs(tpe):
    return object_enemies[tpe]
def _getActors(tpe, objects):
    return [a for a in enemies[tpe].values() if a['object_fn'] in objects]
    # Return all the actors associated with those objects that match the given type
def _getAvailableActors(actor, possible_objects):
    t = _getActorType(actor)
    return [(a['actor_fn'], a['variables'], a['enemy']) for a in _getActors(t, possible_objects)] 
def _writeEnemySpoilers(spoilers, fn):
    fn = os.path.join(LOGDIR, fn)
    with open(fn, "w") as f:
        for name, setup in spoilers.items():
            if len(setup['objects']) > 0 or len(setup['actors']) > 0:
                f.write('{}\n'.format(name))
            if len(setup['objects']) > 0:
                f.write(' objects\n')
            for o in setup['objects']:
                f.write('  {} -> {}\n'.format(o['old'], o['new']))
            if len(setup['actors']) > 0:
                f.write(' actors\n')
            for a in setup['actors']:
                f.write('  {} [{}] -> {} [{}] ({})\n'.format(a['old'], a['old_var'], a['new'], a['new_var'], a['descr']))
    pass # Write out a formatted spoiler log

spoilers = {}
for r, room in rooms.items():
    for n, setup in room.setups.items():
        for actor in setup['actors']:
            if not isEnemyActor(actor.filename):
                continue
            i = available_actors.index(actor.filename)
            actor.setObject(available_objects[i])
        setup_spoiler = {'objects':[],'actors':[]}
        _getRoomName = lambda r,n: '{}-{} ({})'.format(r,n,room.descr)
#         print('{}-{}'.format(r,n))
        spoilers[_getRoomName(r,n)] = setup_spoiler
        possible_objects = []
        for obj in setup['objects']:
            if not isEnemyObject(obj.filename):
                continue
            tpe = _getObjType(obj, setup['actors'], (r,n)) # Function will need to look at the related actor to see what the type is
            if not tpe:
                continue
            new_obj = random.choice(_getObjs(tpe)) if not selected_enemy else selected_enemy['object_fn']
            room.replaceObject(obj.filename, new_obj, n)
            possible_objects.append(new_obj)
            setup_spoiler['objects'].append({'old':obj.filename, 'new':new_obj})
        for actor in setup['actors']:
            # I have to figure out a way to find the index
        
            index = [i for i in setup['actors'] if i.filename == actor.filename].index(actor)
            
            var_str = '{}-{}'.format(actor.filename,actor.var)
            if not (isEnemyActor(actor.filename) and var_str in list(actor_var_pairs.keys())+list(from_actor_var_pairs.keys())) :
                if isEnemyActor(actor.filename):
#                     if room.fn == 'Bmori1_room_6':
#                         raise Exception()
                        

                    pass#print("Excluding {}/{} in room {} setup # {}".format(actor.filename, actor.var, room.fn, n))
                continue # Only randomize enemies that have defined variables 

            choices = _getAvailableActors(actor, possible_objects)
            if len(choices) == 0:
                continue # This should account for cases where 1 actor in a room can't be randomized but other's can
                # Because otherwise the object would get changed out and then the one that didn't get randomized won't spawn

            choice = random.choice(choices)# ( actor, var_list)
            new_actor = choice[0] if not selected_enemy else selected_enemy['fn']
            new_var = random.choice(choice[1]) if not selected_enemy else selected_enemy['var'] # (varid, descr)
            setup_spoiler['actors'].append({'old': actor.filename, 'new': new_actor, 'old_var': actor.var, 'new_var': new_var, 'descr': new_actor[2]})
            room.replaceActor(actor.filename, new_actor, new_var=new_var[0], index=index, setup=n, replaceObject=False)
print("Rooms Randomized, {} sec".format(time.time()-start_time))

# Optionally set the spawn location
spawnAt(0x3)

# Print out spoilers
_writeEnemySpoilers(spoilers, "spoiler.log")
print("Replacement Log Written".format(time.time()-start_time))

writeData()
print("New ROM Written {}".format(time.time()-start_time))

Seed: 99991542915092.48
1901, 1A01, FF02, FF03
0002, FFFF
enemy-type lists created: 0.01033782958984375
Data read, 0.08135604858398438 sec
Room Info Generated, 2.135327100753784 sec
Rooms Randomized, 2.941145896911621 sec
Replacement Log Written
Writing roms/NEWZOOT.z64
New ROM Written 2.993652105331421


In [ ]:
print(rooms["Bmori1_room_6"].getInfo())

In [ ]:
print(rooms["spot05_room_0"].getInfo())

In [ ]:
# Creates a vanilla rom
readData()
rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
spawnAt(0x56)
writeData()

In [ ]:
readData()
replaceActorInRoom('En_Wf', 'En_Bubble', 'spot05_room_0', var='1F01')
spawnAt(0x56)
writeData()

In [ ]:
spawnAt(0xFC)